In [1]:
import uproot

the file here is a part of `mc23_13p6TeV.601229.PhPy8EG_A14_ttbar_hdamp258p75_SingleLep.deriv.DAOD_PHYSLITE.e8514_s4162_r14622_p6026`:
```
/data/alheld/200gbps-atlas/mc23_13p6TeV.601229.PhPy8EG_A14_ttbar_hdamp258p75_SingleLep.deriv.DAOD_PHYSLITE.e8514_s4162_r14622_p6026/DAOD_PHYSLITE.37223155._000001.pool.root.1
```

In [2]:
! ls -lh /data/alheld/200gbps-atlas/mc23_13p6TeV.601229.PhPy8EG_A14_ttbar_hdamp258p75_SingleLep.deriv.DAOD_PHYSLITE.e8514_s4162_r14622_p6026/DAOD_PHYSLITE.37223155._000001.pool.root.1

-rw-r--r-- 1 alheld atlas-af 530M Mar 29 13:07 /data/alheld/200gbps-atlas/mc23_13p6TeV.601229.PhPy8EG_A14_ttbar_hdamp258p75_SingleLep.deriv.DAOD_PHYSLITE.e8514_s4162_r14622_p6026/DAOD_PHYSLITE.37223155._000001.pool.root.1


In [3]:
fname = "/data/alheld/200gbps-atlas/mc23_13p6TeV.601229.PhPy8EG_A14_ttbar_hdamp258p75_SingleLep.deriv.DAOD_PHYSLITE.e8514_s4162_r14622_p6026/DAOD_PHYSLITE.37223155._000001.pool.root.1"
t = uproot.open({fname: "CollectionTree"})

size_per_key_in_MB = {}
cur_size_in_MB = t.file.source.num_requested_bytes/1000**2  # non-zero at the start (some metadata read)

for key in t.keys():
    # if not any([key.startswith(v) for v in ["AnalysisJetsAuxDyn", "AnalysisLargeRJetsAuxDyn", "AnalysisElectronsAuxDyn", "AnalysisMuonsAuxDyn", "AnalysisTauJetsAuxDyn", "AnalysisPhotonsAuxDyn"]]):
    #     continue

    try:
        t.arrays([key])
        print("read branch", key)
        new_size_in_MB = t.file.source.num_requested_bytes/1000**2
        size_per_key_in_MB[key] = new_size_in_MB - cur_size_in_MB
        cur_size_in_MB = new_size_in_MB
    except:
        print("cannot read branch", key)
    
size_per_key_in_MB

cannot read branch xTrigDecisionAux.
cannot read branch xTrigDecisionAux./xTrigDecisionAux.xAOD::AuxInfoBase
read branch xTrigDecisionAux./xTrigDecisionAux.smk
read branch xTrigDecisionAux./xTrigDecisionAux.bgCode
read branch xTrigDecisionAux./xTrigDecisionAux.tav
read branch xTrigDecisionAux./xTrigDecisionAux.tap
read branch xTrigDecisionAux./xTrigDecisionAux.tbp
read branch xTrigDecisionAux./xTrigDecisionAux.lvl2ErrorBits
read branch xTrigDecisionAux./xTrigDecisionAux.efErrorBits
read branch xTrigDecisionAux./xTrigDecisionAux.lvl2Truncated
read branch xTrigDecisionAux./xTrigDecisionAux.efTruncated
read branch xTrigDecisionAux./xTrigDecisionAux.lvl2PassedPhysics
read branch xTrigDecisionAux./xTrigDecisionAux.efPassedPhysics
read branch xTrigDecisionAux./xTrigDecisionAux.lvl2PassedRaw
read branch xTrigDecisionAux./xTrigDecisionAux.efPassedRaw
read branch xTrigDecisionAux./xTrigDecisionAux.lvl2PassedThrough
read branch xTrigDecisionAux./xTrigDecisionAux.efPassedThrough
read branch xTrig

{'xTrigDecisionAux./xTrigDecisionAux.smk': 0.030205000000000037,
 'xTrigDecisionAux./xTrigDecisionAux.bgCode': 0.008187999999999973,
 'xTrigDecisionAux./xTrigDecisionAux.tav': 0.774954,
 'xTrigDecisionAux./xTrigDecisionAux.tap': 0.7749539999999999,
 'xTrigDecisionAux./xTrigDecisionAux.tbp': 0.7749540000000001,
 'xTrigDecisionAux./xTrigDecisionAux.lvl2ErrorBits': 0.02048799999999984,
 'xTrigDecisionAux./xTrigDecisionAux.efErrorBits': 0.02018399999999998,
 'xTrigDecisionAux./xTrigDecisionAux.lvl2Truncated': 0.008636000000000088,
 'xTrigDecisionAux./xTrigDecisionAux.efTruncated': 0.00850799999999996,
 'xTrigDecisionAux./xTrigDecisionAux.lvl2PassedPhysics': 0.08133599999999985,
 'xTrigDecisionAux./xTrigDecisionAux.efPassedPhysics': 1.1264279999999998,
 'xTrigDecisionAux./xTrigDecisionAux.lvl2PassedRaw': 0.0808750000000007,
 'xTrigDecisionAux./xTrigDecisionAux.efPassedRaw': 1.1262879999999997,
 'xTrigDecisionAux./xTrigDecisionAux.lvl2PassedThrough': 0.0813360000000003,
 'xTrigDecisionAux./x

In [4]:
print(f"all branches: {sum(size_per_key_in_MB.values())} MB")  # doesn't include initial metadata read

all branches: 554.5054250000001 MB


In [5]:
import os
file_size = os.path.getsize(fname) / 1000**2
file_size

554.994903

In [6]:
t.file.source.num_requested_bytes/1000**2

555.336994

sanity check: looks like we read pretty much everything, seems good

In [7]:
import json

size_per_key_in_MB["_file_size"] = file_size

with open("branch_sizes.json", "w") as f:
    f.write(json.dumps(size_per_key_in_MB, sort_keys=True, indent=4))

## sort branches, find branches to hit 10% read

In [8]:
import pathlib

size_per_key_in_MB = json.loads(pathlib.Path("branch_sizes.json").read_text())
file_size = size_per_key_in_MB["_file_size"]

list_of_branches_to_read = []
size_of_branches = 0

# veto the metadata branch and also possibly problematic ones that could vary a lot across files
branches_to_veto = ["_file_size", "AnalysisJetsAuxDyn.GhostTrack", "AnalysisJetsAuxDyn.btaggingLink"] #, "HLTNav_Summary_DAODSlimmedAuxDyn.decisions", "InDetTrackParticlesAuxDyn.definingParametersCovMatrixOffDiag"]

for k, v in reversed(sorted(size_per_key_in_MB.items(), key=lambda it: it[1])):
    # only keep branches relating to specific physics objects
    if not any([vv in k for vv in ["AnalysisJetsAuxDyn", "AnalysisElectronsAuxDyn", "AnalysisMuonsAuxDyn"]]):
        continue
    
    if k in branches_to_veto:
        continue

    list_of_branches_to_read.append(k)
    size_of_branches += v
    if size_of_branches / file_size > 0.10:  # target: 25%
        break

for b in list_of_branches_to_read:
    print(f"{b}: {size_per_key_in_MB[b] / file_size:.2%}")

AnalysisJetsAuxDyn.EnergyPerSampling: 1.56%
AnalysisJetsAuxDyn.SumPtTrkPt500: 1.49%
AnalysisJetsAuxDyn.TrackWidthPt1000: 0.83%
AnalysisJetsAuxDyn.NumTrkPt500: 0.49%
AnalysisJetsAuxDyn.NumTrkPt1000: 0.36%
AnalysisJetsAuxDyn.SumPtChargedPFOPt500: 0.30%
AnalysisJetsAuxDyn.Timing: 0.23%
AnalysisJetsAuxDyn.JetConstitScaleMomentum_eta: 0.23%
AnalysisJetsAuxDyn.ActiveArea4vec_eta: 0.23%
AnalysisJetsAuxDyn.DetectorEta: 0.23%
AnalysisJetsAuxDyn.eta: 0.23%
AnalysisJetsAuxDyn.JetConstitScaleMomentum_phi: 0.23%
AnalysisJetsAuxDyn.ActiveArea4vec_phi: 0.23%
AnalysisJetsAuxDyn.phi: 0.23%
AnalysisJetsAuxDyn.JetConstitScaleMomentum_m: 0.22%
AnalysisJetsAuxDyn.JetConstitScaleMomentum_pt: 0.22%
AnalysisJetsAuxDyn.Width: 0.22%
AnalysisJetsAuxDyn.EMFrac: 0.22%
AnalysisJetsAuxDyn.pt: 0.22%
AnalysisJetsAuxDyn.m: 0.22%
AnalysisJetsAuxDyn.ActiveArea4vec_m: 0.21%
AnalysisJetsAuxDyn.ActiveArea4vec_pt: 0.21%
AnalysisJetsAuxDyn.DFCommonJets_QGTagger_TracksWidth: 0.19%
AnalysisJetsAuxDyn.JVFCorr: 0.17%
AnalysisJets

In [9]:
list_of_branches_to_read

['AnalysisJetsAuxDyn.EnergyPerSampling',
 'AnalysisJetsAuxDyn.SumPtTrkPt500',
 'AnalysisJetsAuxDyn.TrackWidthPt1000',
 'AnalysisJetsAuxDyn.NumTrkPt500',
 'AnalysisJetsAuxDyn.NumTrkPt1000',
 'AnalysisJetsAuxDyn.SumPtChargedPFOPt500',
 'AnalysisJetsAuxDyn.Timing',
 'AnalysisJetsAuxDyn.JetConstitScaleMomentum_eta',
 'AnalysisJetsAuxDyn.ActiveArea4vec_eta',
 'AnalysisJetsAuxDyn.DetectorEta',
 'AnalysisJetsAuxDyn.eta',
 'AnalysisJetsAuxDyn.JetConstitScaleMomentum_phi',
 'AnalysisJetsAuxDyn.ActiveArea4vec_phi',
 'AnalysisJetsAuxDyn.phi',
 'AnalysisJetsAuxDyn.JetConstitScaleMomentum_m',
 'AnalysisJetsAuxDyn.JetConstitScaleMomentum_pt',
 'AnalysisJetsAuxDyn.Width',
 'AnalysisJetsAuxDyn.EMFrac',
 'AnalysisJetsAuxDyn.pt',
 'AnalysisJetsAuxDyn.m',
 'AnalysisJetsAuxDyn.ActiveArea4vec_m',
 'AnalysisJetsAuxDyn.ActiveArea4vec_pt',
 'AnalysisJetsAuxDyn.DFCommonJets_QGTagger_TracksWidth',
 'AnalysisJetsAuxDyn.JVFCorr',
 'AnalysisJetsAuxDyn.DFCommonJets_QGTagger_TracksC1',
 'AnalysisJetsAuxDyn.PSFrac',


## sanity check: load file again, read these branches, ensure we hit the right fraction of the file

In [10]:
t = uproot.open({fname: "CollectionTree"})

cur_size_in_MB = t.file.source.num_requested_bytes/1000**2  # non-zero at the start (some metadata read)

t.arrays(list_of_branches_to_read)

new_size_in_MB = t.file.source.num_requested_bytes/1000**2
print(cur_size_in_MB, new_size_in_MB)

0.831569 56.42101


In [11]:
(new_size_in_MB - cur_size_in_MB) / file_size

0.10016207482179346